# Total time derivative $\frac{d^n z(t)}{d t^n}$ with `jet`

Given the initial value problem defined by the dynamics
$$ z'(t) = \frac{\partial z}{\partial t} = f(z(t),t) $$
and initial conditions $z_0$ at $t_0$. 

We have the solution $$z(t) = z_0 + \int_{t_0}^t f(z(s),s) ds$$

We are interested in computing the $n$th time derivative of the solution $z(t)$: $$\frac{d^n z(t)}{d t^n}$$

## Autonomous System

To simplify things lets consider an autonomous system, one whose dynamics are time invariant:
$$ z'(t) = \frac{\partial z}{\partial t} = f(z(t),t) = g(z(t)) $$

To allow us to test correctness we consider a simple dynamical system,
$$g(z(t)) = 0.5 z$$
which has a closed form solution
$$z(t) = \exp[0.5t-0.5t_0]*z_0$$

In [31]:
import jax.numpy as np
from jax import grad
from scipy.integrate import odeint

## Initial Value Problem

# Dynamics
def f(z,t):
    return g(z)
def g(z):
    return 0.5*z

# Initial Conditions
t0, z0 = 2., 1.

# Closed form solution
def sol(t):
    return np.exp(0.5*t - 0.5*t0) * z0

# We evaluate our solution at a given time
t_eval = 4.
z_eval = sol(t_eval)

# Confirm the correctness of closed-form
assert np.isclose(odeint(f, z0, [t0, t_eval])[1], z_eval)

## Total time-derivatives of the solution
Since we have access to closed-form solution we can nest jax.grad to compute the nth derivative

In [32]:
def nest(f, n): # why do I have to implement this :(
    def rfun(p):
        return reduce(lambda x, _: f(x), xrange(n), p)
    return rfun

def n_grad(f,x,n):
    return nest(grad,n)(f)(x)

# This is the primal and first 4 derivatives of the solution
n_grads = [n_grad(sol,t_eval,i) for i in range(0,5)]
print n_grads

[DeviceArray(2.7182817, dtype=float32), DeviceArray(1.3591409, dtype=float32), DeviceArray(0.67957044, dtype=float32), DeviceArray(0.33978522, dtype=float32), DeviceArray(0.16989261, dtype=float32)]


## Taylor's Method

Unfortunately, we are rarely interested in systems with known closed-form solutions. Instead we only have access to $f(z,t) = g(z)$ and the current state `z_eval`. We could take gradients through the numerical solve, and for the $n$th order, nest these. But we want to avoid this.

Consider the input state $z_0$, which will be carrying its own higher-order sensitvity terms $(z_1,z_2,\dots, z_d)$. So,

$$ z_0(t) = z_0 +  z_1 t + z_2 t^2  + \cdots + z_d t^d $$

Then Taylor's method gives us an $O(t^d)$ approximation to the solution

$$g(z_0(t)) = v_0 + v_1 t + v_2 t^2 + \cdots + v_d t^d $$

Where 
$$
\begin{align}
v_0 &= g(z_0)\\
v_1 &= g'(z_0) z_1\\
v_2 &= g'(z_0) z_2 + \frac{1}{2} g''(z_0) z_1 z_1\\
v_3 &= g'(z_0) z_3 + g''(z_0) z_1 z_2 + \frac{1}{6} g'''(z_0)z_1 z_1 z_1\\
\dots
\end{align}
$$

Computing the resulting coefficients $v_0 \dots v_d$ is the purpose of `jet`, which shares work in operations common accross the $v_i$s, e.g. sharing $g'(z_0)$.

# Jet

We can use `jet` to compute higher order sensitivities of a function with sensitivity vectors `v`. The `n`th component of the terms returned by jet corresponds to
$$ jet_n(f,x,v) = \frac{\partial^n}{\partial \epsilon^n} f(x + v_1 * \epsilon + v_2 * \frac{\epsilon^2}{2!} + \dots)|_{\epsilon = 0}$$

In [85]:
from jax import jet
import numpy.random as npr
from scipy.special import factorial as fact

def taylor_expansion(f,primals,series):
    def expansion(eps):
        tayterms = [
        sum([eps**(i + 1) * terms[i] / fact(i + 1) for i in range(len(terms))])
        for terms in series
    ]
        return f(*map(sum, zip(primals, tayterms)))
    return expansion

order = 4
vs = list(npr.randn(order))
jet_primal, jet_coefs = jet(g,(z_eval,),(vs,))

print np.isclose(taylor_expansion(g,(z_eval,),(vs,))(0.),jet_primal)
print np.isclose(grad(taylor_expansion(g,(z_eval,),(vs,)))(0.), jet_coefs[0])
print np.isclose(grad(grad(taylor_expansion(g,(z_eval,),(vs,))))(0.), jet_coefs[1])
print np.isclose(grad(grad(grad(taylor_expansion(g,(z_eval,),(vs,)))))(0.),jet_coefs[2])

True
True
True
True


## Jet for Total Derivatives

Calling `jet` with "standard" sentivitiy terms `(1.,0.,0.,...)` corresponds to the expansion

In [121]:
def shift_jet(old_jet):
    return [old_jet[0]] + old_jet[1]


def std_terms(x, n):
    return [np.ones_like(x)] + [np.zeros_like(x)] * (n - 1)


def zero_terms(x, n):
    return [np.zeros_like(x)] * n



print "\n First pass\n"
z_terms = std_terms(z_eval,3)
j1 = jet(g,(z_eval,),(z_terms,))
print z_terms
print j1



print "\n Second pass\n"
z_terms = shift_jet(j1)
j2 = jet(g,(z_eval,),(z_terms,))
print z_terms
print j2

print "\n Third pass\n"
z_terms = shift_jet(j2)
j3 = jet(g,(z_eval,),(z_terms,))
print z_terms
print j3


 First pass

[Const(1.0), Const(0.0), Const(0.0)]
(DeviceArray(1.3591409, dtype=float32), [DeviceArray(0.5, dtype=float32), DeviceArray(0., dtype=float32), DeviceArray(0., dtype=float32)])

 Second pass

[DeviceArray(1.3591409, dtype=float32), DeviceArray(0.5, dtype=float32), DeviceArray(0., dtype=float32), DeviceArray(0., dtype=float32)]
(DeviceArray(1.3591409, dtype=float32), [DeviceArray(0.67957044, dtype=float32), DeviceArray(0.25, dtype=float32), DeviceArray(0., dtype=float32), DeviceArray(0., dtype=float32)])

 Third pass

[DeviceArray(1.3591409, dtype=float32), DeviceArray(0.67957044, dtype=float32), DeviceArray(0.25, dtype=float32), DeviceArray(0., dtype=float32), DeviceArray(0., dtype=float32)]
(DeviceArray(1.3591409, dtype=float32), [DeviceArray(0.67957044, dtype=float32), DeviceArray(0.33978522, dtype=float32), DeviceArray(0.125, dtype=float32), DeviceArray(0., dtype=float32), DeviceArray(0., dtype=float32)])


In [123]:
n_grads

[DeviceArray(2.7182817, dtype=float32),
 DeviceArray(1.3591409, dtype=float32),
 DeviceArray(0.67957044, dtype=float32),
 DeviceArray(0.33978522, dtype=float32),
 DeviceArray(0.16989261, dtype=float32)]